In [3]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "final"

filter_name = "vit_b"
filter_type = "vo"
filter_state = "finished"

filter_metric = "Objective/eval_all"
filter_operation = "min"

metrics = ["Objective/eval_forward", "Objective/eval_left", "Objective/eval_right", "eval_all_unified_depth/aux_depth_loss",
           "Objective/train"]

configs = ["config.VO.MODEL.visual_backbone", "config.VO.MODEL.pretrain_backbone",
           "config.VO.MODEL.cls_action", "config.VO.TRAIN.depth_aux_loss"]

step_threshold = 150

out_name = "experiments_2020_vit_b"

In [5]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api(timeout=45)
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, filter_metric, *metrics])

for run in tqdm.tqdm(runs):
    
    if filter_state and run._state != filter_state:
        continue
    
    if filter_type and filter_type not in run.name:
        continue
        
    if filter_name and filter_name not in run.name:
        continue
    
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    select = run_history["_step"]<step_threshold
    
    if filter_operation == "min":
        filter_metric_idx = run_history[filter_metric][select].argmin()
    elif filter_operation == "max":
        filter_metric_idx = run_history[filter_metric][select].argmax()
    else:
        break
    run_dict["best_run_path"] = run.config["config"]["CHECKPOINT_FOLDER"] + f"/ckpt_epoch_{filter_metric_idx}.pth"
    run_dict["best_run_idx"] = filter_metric_idx
        
    run_dict[filter_metric] = run_history[filter_metric][filter_metric_idx]
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        # catch models that only evaluate some actions
        if metric in run_history.keys():
            run_dict[metric] = run_history[metric][filter_metric_idx]
        else:
            run_dict[metric] = -1
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df

 29%|████████████████████████████████████████████████████▍                                                                                                                              | 24/82 [00:00<00:00, 101.94it/s]


NameError: name 'filter_metric_idx' is not defined

In [ ]:
dt = datetime.datetime.now().strftime("%Y%m%d")
filename = f"{out_name}_{dt}"

In [16]:
# # save df as csv file
# extension = ".csv"
# runs_df.to_csv(filename+extension)

In [5]:
!apt-get update -y
!apt-get install -y python-xlsxwriter
!pip install openpyxl

Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,575 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Err:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main 

     |████████████████████████████████| 242 kB 3.7 MB/s            


In [6]:
# save df as excel file
extension = ".xlsx"
with pd.ExcelWriter(filename+extension) as excel_writer:
    runs_df.to_excel(excel_writer, sheet_name='Sheet1')